In [11]:
#pip install cassandra-driver
#pip install overpy


combine different index

In [3]:
# import math
# radius = 5000
# center_lat = 52.520008
# center_lng = 13.404954
# find bounding box
# radius_lat = radius / 111.0
# radius_lng = radius / (111.0 * math.cos(center_lat * math.pi / 180.0))
# min_lat = center_lat - radius_lat
# max_lat = center_lat + radius_lat
# min_lng = center_lng - radius_lng
# max_lng = center_lng + radius_lng
# find all shops in bounding box
# result = api.query("""
#     node["shop"](around:5000,52.520008,13.404954);
#     out;
#     """)
# find all shops in germany

result = api.query("""
    node["shop"](50.746,7.154,52.748,9.157);
    out;
    """)




<overpy.Node id=30319482 lat=51.6098648 lon=7.1806063>


In [8]:
import math
import overpy
radius = 5.0
center_lat = 52.520008
center_lng = 13.404954
#find bounding box
radius_lat = radius / 111.0
radius_lng = radius / (111.0 * math.cos(center_lat * math.pi / 180.0))
min_lat = center_lat - radius_lat
max_lat = center_lat + radius_lat
min_lng = center_lng - radius_lng
max_lng = center_lng + radius_lng
#find all shops in bounding box
api = overpy.Overpass()
result = api.query("""
    node["shop"](around:5000,52.520008,13.404954);
    out;
    """)


In [ ]:
import folium

# create map
map_osm = folium.Map(location=[52.520008, 13.404954], zoom_start=13)
# add markers to map
for node in result.nodes:
    folium.Marker([node.lat, node.lon], popup=node.tags.get("name", "")).add_to(map_osm)
map_osm

In [9]:
for node in result.nodes:
    print("Name: %s" % node.tags.get("name", "n/a"))
    print("  Type: %s" % node.tags.get("shop", "n/a"))
    print("  Latitude: %f" % float(node.lat))


Name: Netto Marken-Discount
  Type: supermarket
  Latitude: 52.509819
Name: n/a
  Type: bakery
  Latitude: 52.510360
Name: dm
  Type: chemist
  Latitude: 52.510458
Name: Berlin Pavillon
  Type: gift
  Latitude: 52.517604
Name: Vintage Revivals
  Type: clothes
  Latitude: 52.524102
Name: Antiquariat am Gendarmenmarkt
  Type: antiques
  Latitude: 52.514106
Name: Solid Earth
  Type: copyshop
  Latitude: 52.536293
Name: Netto
  Type: supermarket
  Latitude: 52.551506
Name: Kauf Markt9
  Type: convenience
  Latitude: 52.511047
Name: EDEKA Thaut
  Type: supermarket
  Latitude: 52.510061
Name: Back-Factory
  Type: bakery
  Latitude: 52.486277
Name: Öz-Gida
  Type: supermarket
  Latitude: 52.487314
Name: Lutz Paul BrillenmacherBerlin
  Type: optician
  Latitude: 52.512658
Name: Repro Kopie
  Type: copyshop
  Latitude: 52.511982
Name: n/a
  Type: vacant
  Latitude: 52.510475
Name: EDEKA
  Type: supermarket
  Latitude: 52.515615
Name: hd-computer
  Type: computer
  Latitude: 52.488047
Name: JYSK

In [ ]:

# query = $"[out:json];node({cityMin.y},{cityMin.x},{cityMax.y},{cityMax.x});(node[amenity=mall]({cityMin.y},{cityMin.x},{cityMax.y},{cityMax.x});node[leisure=park]({cityMin.y},{cityMin.x},{cityMax.y},{cityMax.x});node[amenity=restaurant]({cityMin.y},{cityMin.x},{cityMax.y},{cityMax.x});node[shop]({cityMin.y},{cityMin.x},{cityMax.y},{cityMax.x});node[leisure=stadium]({cityMin.y},{cityMin.x},{cityMax.y},{cityMax.x}););out;";

In [ ]:
for node in result.nodes:
    print("Name: %s" % node.tags.get("name", "n/a"))
    print("  Type: %s" % node.tags.get("shop", "n/a"))
    print("  Latitude: %f" % float(node.lat))

In [21]:
list_lat = []
list_lng = []
list_city = []
list_place = []
for node in result.nodes:
    list_lat.append(node.lat)
    list_lng.append(node.lon)
    list_city.append(node.tags.get("addr:city", "n/a"))
    list_place.append(node.tags.get("name", "n/a"))

    

In [28]:
len(list_lat)

38573

In [27]:
# add this lists to dataframe
df = pd.DataFrame(columns=['lat', 'lng', 'city', 'place'])
for i in range(len(list_lat)):
    # add this lists to dataframe
    new_data = {'lat' : list_lat[i], 'lng' : list_lng[i], 'city' : list_city[i], 'place' : list_place[i]}
    new_data_df = pd.DataFrame(new_data, index=[0])
    df = pd.concat([df, new_data_df], ignore_index=True)


In [38]:
# add new column type with 0.9 common eggs and 0.1 rare eggs
import numpy as np
df['type'] = np.random.choice(['common', 'rare'], df.shape[0], p=[0.9, 0.1])

In [39]:
# drop location column
df.head()

,lat,lng,city,place,type
0,51.6098648,7.1806063,Recklinghausen,Lidl,common
1,51.4252831,7.1584139,Bochum,n/a,common
2,51.7322883,8.7648051,Paderborn,Aral,common
3,51.3978427,7.1545747,Hattingen,Der Schultenhof - Ökologische Landwirtschaft,rare
4,52.1984850,8.7176659,Löhne,Sieveking,common


In [1]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["treasure_hunt"]
collection = db["eggs_2"]

In [42]:
# insert dataframe to mongodb
# cast lat and lng to float
df['lat'] = df['lat'].astype(float)
df['lng'] = df['lng'].astype(float)
# cast city and place and type to string
df['city'] = df['city'].astype(str)
df['place'] = df['place'].astype(str)
df['type'] = df['type'].astype(str)


In [44]:
# convert dataframe to dictionary
data_dict = df.to_dict("records")

In [45]:
# insert data_dict to mongodb
collection.insert_many(data_dict)

In [2]:
# show  5 documents in eggs2 collection
for x in collection.find().limit(5):
    print(x)

{'_id': ObjectId('65202581825d78924c8346bc'), 'lat': 51.6098648, 'lng': 7.1806063, 'city': 'Recklinghausen', 'place': 'Lidl', 'type': 'common', 'location': {'type': 'Point', 'coordinates': [51.6098648, 7.1806063]}}
{'_id': ObjectId('65202581825d78924c8346bd'), 'lat': 51.4252831, 'lng': 7.1584139, 'city': 'Bochum', 'place': 'n/a', 'type': 'common', 'location': {'type': 'Point', 'coordinates': [51.4252831, 7.1584139]}}
{'_id': ObjectId('65202581825d78924c8346be'), 'lat': 51.7322883, 'lng': 8.7648051, 'city': 'Paderborn', 'place': 'Aral', 'type': 'common', 'location': {'type': 'Point', 'coordinates': [51.7322883, 8.7648051]}}
{'_id': ObjectId('65202581825d78924c8346bf'), 'lat': 51.3978427, 'lng': 7.1545747, 'city': 'Hattingen', 'place': 'Der Schultenhof - Ökologische Landwirtschaft', 'type': 'rare', 'location': {'type': 'Point', 'coordinates': [51.3978427, 7.1545747]}}
{'_id': ObjectId('65202581825d78924c8346c0'), 'lat': 52.198485, 'lng': 8.7176659, 'city': 'Löhne', 'place': 'Sieveking', 

In [2]:
#number of records in eggs2 collection
collection.count_documents({})

38573

In [3]:
# find all indexes in eggs2 collection
collection.index_information()


{'_id_': {'v': 2, 'key': [('_id', 1)]}}

using timeit to measure time
n means number of times
r means repeat

In [7]:
# 
collection.find({"city":"Bochum"}).explain()

{'explainVersion': '2',
 'queryPlanner': {'namespace': 'treasure_hunt.eggs_2',
  'indexFilterSet': False,
  'parsedQuery': {'city': {'$eq': 'Bochum'}},
  'queryHash': '0C454993',
  'planCacheKey': '5758F65A',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'queryPlan': {'stage': 'COLLSCAN',
    'planNodeId': 1,
    'filter': {'city': {'$eq': 'Bochum'}},
    'direction': 'forward'},
   'slotBasedPlan': {'slots': '$$RESULT=s5 env: { s1 = TimeZoneDatabase(America/Noronha...Asia/Taipei) (timeZoneDB), s3 = 1696928957501 (NOW), s2 = Nothing (SEARCH_META), s7 = "Bochum" }',
    'stages': '[1] filter {traverseF(s4, lambda(l1.0) { ((l1.0 == s7) ?: false) }, false)} \n[1] scan s5 s6 none none none none lowPriority [s4 = city] @"26575093-07b1-49c3-ae53-25f440fd9c50" true false '}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 1622,
  'executionTimeMillis': 118,
  'tota

In [ ]:
# make sharding key for cities on mongodb col



In [14]:
%%timeit -n 100 -r 2
# find eggs in Bochum
random_city = np.random.choice(cities)
collection.find({'city': random_city});
count_eggs = collection.count_documents({'city': random_city})
print("Number of eggs in " + random_city + " is " + str(count_eggs))
  

Number of eggs in Neuenrade is 12
Number of eggs in Verl is 106
Number of eggs in Hilchenbach is 5
Number of eggs in Landesbergen is 6
Number of eggs in Freren is 2
Number of eggs in Bad Driburg is 63
Number of eggs in Dülmen is 70
Number of eggs in Bissendorf is 9
Number of eggs in Mellinghausen is 2
Number of eggs in Bünde is 44
Number of eggs in Netphen is 22
Number of eggs in Werne is 22
Number of eggs in Hagen am Teutoburger Wald is 18
Number of eggs in Edertal is 4
Number of eggs in Enger is 32
Number of eggs in Anröchte is 9
Number of eggs in Brilon is 30
Number of eggs in Lübbecke is 117
Number of eggs in Dörentrup is 11
Number of eggs in Harsewinkel is 26
Number of eggs in Meerbeck is 3
Number of eggs in Ense is 10
Number of eggs in Cölbe is 4
Number of eggs in Liebenau is 9
Number of eggs in Ochtrup is 64
Number of eggs in Bad Driburg is 63
Number of eggs in Anröchte is 9
Number of eggs in Löningen is 120
Number of eggs in Sundern is 1
Number of eggs in Borchen is 64
Number o

In [50]:
# drop location column
# collection.update_many({}, {'$unset': {'location':1}})

In [59]:
%%timeit -n 100 -r 2
# query mongodb
# find all  eggs in Bonn
result = collection.find({'city':'Bonn'})



5.24 µs ± 1.78 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)


In [ ]:
# know size of processed data in mongodb 


In [58]:
# find distinct city
collection.distinct('city')

['Ahlen',
 'Alfhausen',
 'Allendorf (Eder)',
 'Altena',
 'Altenbeken',
 'Altenberge',
 'Amöneburg',
 'Angelburg',
 'Ankum',
 'Anröchte',
 'Arnsberg',
 'Ascheberg',
 'Attendorn',
 'Auetal',
 'Augustdorf',
 'Bad Arolsen',
 'Bad Bentheim',
 'Bad Berleburg',
 'Bad Driburg',
 'Bad Eilsen',
 'Bad Endbach',
 'Bad Essen',
 'Bad Iburg',
 'Bad Laasphe',
 'Bad Laer',
 'Bad Lippspringe',
 'Bad Oeynhausen',
 'Bad Rothenfelde',
 'Bad Salzuflen',
 'Bad Sassendorf',
 'Bad Wildungen',
 'Bad Wünnenberg',
 'Badbergen',
 'Bahrenborstel',
 'Bakum',
 'Balve',
 'Barenburg',
 'Barnstorf',
 'Barntrup',
 'Beckum',
 'Beelen',
 'Belm',
 'Berge',
 'Bergisch Gladbach',
 'Bergkamen',
 'Bergneustadt',
 'Bersenbrück',
 'Bestwig',
 'Betzdorf',
 'Biedenkopf',
 'Bielefeld',
 'Billerbeck',
 'Bippen',
 'Bissendorf',
 'Blomberg',
 'Bochum',
 'Bohmte',
 'Bonn',
 'Borchen',
 'Borgholzhausen',
 'Borstel',
 'Bottenhorn',
 'Brakel',
 'Bramsche',
 'Breckerfeld',
 'Breidenbach',
 'Brilon',
 'Brockum',
 'Buchenau',
 'Buchholz',
 'B

In [52]:
collection.update_many({}, [
    {
        "$set": {
            "location": {
                "type": "Point",
                "coordinates": ["$lat", "$lng"]  # Store lat and lng as [lng, lat]
            }
        }
    }
])

In [36]:
# create map
import folium

map_osm = folium.Map(location=[center_lat, center_lng], zoom_start=13)
# add a marker for every shop
for i in range(len(list_lat)):
    folium.Marker([list_lat[i], list_lng[i]], popup=list_city[i]).add_to(map_osm)

In [ ]:
# print 10k shop in germany
# result = api.query("""
#     node["shop"](around:10000,52.520008,13.404954);
#     out;
#     """)
# for node in result.nodes:
#     print("Name: %s" % node.tags.get("name", "n/a"))
#     print("  Lat: %f, Lon: %f" % (node.lat, node.lon))

In [10]:
# retrieve 
cityname = "Cologne"
radius = 1000
centerlat = 50.948
centerlng = 6.927
# find boundry of city
# result = api.query("""[out:json];area[name="Cologne"];out;""")

# fetch random places in a circle 6km  around a center point
result = api.query("""
    [out:json];
    area["place"="Cologne"](around:5000,50.948,6.927);
    out;
    """)


# find random places in a circle 6km  around a center point and store in list


In [11]:
for row in result.nodes:
    print(row)

In [ ]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];area[name="Germany"];(node[place="village"](area););out;
"""

response = requests.get(
    overpass_url, 
    params={'data': overpass_query}
)

coords = []
if response.status_code == 200:
    data = response.json()
    places = data.get('elements', [])
    for place in places:
        coords.append((place['lat'], place['lon']))
     print ("Got %s village coordinates!" % len(coords))
     print (coords[0])
else:
     print("Error")